In [1]:
# USAGE
# python fashion_mnist.py

# set the matplotlib backend so figures can be saved in the background
import matplotlib

# import the necessary packages
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.normalization import BatchNormalization
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers.core import Activation, Flatten, Dropout, Dense

matplotlib.use("Agg")
# import the necessary packages
from PIL import Image
# import pickle
# import cloudpickle
# import _pickle as cpickle
from sklearn.metrics import classification_report
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.datasets import fashion_mnist
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras import backend as K
from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os


# layer


def build(width, height, depth, classes):
    global layer
    # initialize the model along with the input shape to be
    # "channels last" and the channels dimension itself
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    # if we are using "channels first", update the input shape
    # and channels dimension
    if K.image_data_format() == "channels_first":
        inputShape = (depth, height, width)
        chanDim = 1

    # first CONV => RELU => CONV => RELU => POOL layer set
    model.add(Conv2D(32, (3, 3), padding="same",
                     input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    
    model.add(Conv2D(32, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # second CONV => RELU => CONV => RELU => POOL layer set
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # first (and only) set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # softmax classifier
    model.add(Dense(classes))
    layer = Activation("softmax")
    model.add(layer)
    print(layer.get_weights())
    # return the constructed network architecture
    return model


def train(trainX, trainY, testX, testY, labelData):
    # initialize the number of epochs to train for, base learning rate,
    # and batch size
    NUM_EPOCHS = 5
    INIT_LR = 1e-2
    BS = 32

    # initialize the optimizer and model
    print("[INFO] compiling model...")
    opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS)
    model = build(width=28, height=28, depth=1, classes=10)
    model.compile(loss="categorical_crossentropy", optimizer=opt,
                  metrics=["accuracy"])

    # train the network
    print("[INFO] training model...")
    H = model.fit(trainX, trainY,
                  validation_data=(testX, testY),
                  batch_size=BS, epochs=NUM_EPOCHS)
    # save model
    # pickle_out = open("model.pickle", "wb")
    # pickle.dump(model, pickle_out)
    # pickle_out.close()

    # make predictions on the test set
    preds = model.predict

    # show a nicely formatted classification report
    # print("[INFO] evaluating network...")
    # print(classification_report(testY.argmax(axis=1), preds.argmax(axis=1),
    #                          target_names=labelData))

    # makeOutfits()
    # plot the training loss and accuracy
    N = NUM_EPOCHS
    # plt.style.use("ggplot")
    # plt.figure()
    # plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    # plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    # plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
    # plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
    # plt.title("Training Loss and Accuracy on Dataset")
    # plt.xlabel("Epoch #")
    # plt.ylabel("Loss/Accuracy")
    # plt.legend(loc="lower left")
    # plt.savefig("plot.png")

    return model
    # for lay in model.layers:
    #    print(lay.name)
    #    print(lay.get_weights())
    # cv2.waitKey(0)


def getData():
    print("[INFO] loading Fashion MNIST...")

    ((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()

    # if we are using "channels first" ordering, then reshape the design
    # matrix such that the matrix is:
    # 	num_samples x depth x rows x columns
    if K.image_data_format() == "channels_first":
        trainX = trainX.reshape((trainX.shape[0], 1, 28, 28))
        testX = testX.reshape((testX.shape[0], 1, 28, 28))

    # otherwise, we are using "channels last" ordering, so the design
    # matrix shape should be: num_samples x rows x columns x depth
    else:
        trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
        testX = testX.reshape((testX.shape[0], 28, 28, 1))

    # scale data to the range of [0, 1]
    trainX = trainX.astype("float32") / 255.0
    testX = testX.astype("float32") / 255.0

    # one-hot encode the training and testing labels
    trainY = np_utils.to_categorical(trainY, 10)
    testY = np_utils.to_categorical(testY, 10)

    labelNames = ["top", "trouser", "pullover", "dress", "coat",
                  "sandal", "shirt", "sneaker", "bag", "ankle boot"]
    return trainX, trainY, testX, testY, labelNames


def makeOutfits(trainX, trainY, testX, testY, labelData, model):
    images = []
    i = 0
    # randomly select a few testing fashion items
    for i in np.random.choice(np.arange(0, len(testY)), size=(100,)):
        # classify the clothing
        probs = model.predict(testX[np.newaxis, i])
        prediction = probs.argmax(axis=1)
        label = labelData[prediction[0]]
        print(label)
        # extract the image from the testData if using "channels_first"
        # ordering
        if K.image_data_format() == "channels_first":
            image = (testX[i][0] * 255).astype("uint8")

        # otherwise we are using "channels_last" ordering
        else:
            image = (testX[i] * 255).astype("uint8")

        image = cv2.merge([image] * 3)
        image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
        color = (0, 255, 0)
        cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)
        images.append(image)
        if label == "top" or label == "shirt":
            # extract the image from the testData if using "channels_first"
            # ordering
            if K.image_data_format() == "channels_first":
                image = (testX[i][0] * 255).astype("uint8")

            # otherwise we are using "channels_last" ordering
            else:
                image = (testX[i] * 255).astype("uint8")

            image = cv2.merge([image] * 3)
            image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
            images.append(image)
            print("IN IF")
            for j in np.random.choice(np.arange(0, len(testY)), size=(100,)):
                print("IN FOR")
                # classify the clothing
                probs = model.predict(testX[np.newaxis, j])
                prediction = probs.argmax(axis=1)
                labelT = labelData[prediction[0]]
                if labelT == "trouser":
                    i = i + 1
                    print("FOUND TROUSER")
                    # extract the image from the testData if using "channels_first"
                    # ordering
                    if K.image_data_format() == "channels_first":
                        image = (testX[i][0] * 255).astype("uint8")

                    # otherwise we are using "channels_last" ordering
                    else:
                        image = (testX[i] * 255).astype("uint8")

                    image = cv2.merge([image] * 3)
                    image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
                    color = (0, 255, 0)
                    cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)
                    # add the image to our list of output images
                    images.append(image)

                    # construct the montage for the images
                    montage = build_montages(images, (96, 96), (4, 4))[0]

                    # show the output montage
                    path = os.getcwd() + "\\" + str(i) + "TrouserTop.png"
                    cv2.imwrite(path, montage)
                    images = []
                    montage = []
                break

                # cv2.imshow("Fashion MNIST", montage)
        if label == "dress":
            # extract the image from the testData if using "channels_first"
            # ordering
            if K.image_data_format() == "channels_first":
                image = (testX[i][0] * 255).astype("uint8")

            # otherwise we are using "channels_last" ordering
            else:
                image = (testX[i] * 255).astype("uint8")

            image = cv2.merge([image] * 3)
            image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
            images.append(image)
            for j in np.random.choice(np.arange(0, len(testY)), size=(100,)):
                # classify the clothing
                probs = model.predict(testX[np.newaxis, j])
                prediction = probs.argmax(axis=1)
                labelT = labelData[prediction[0]]
                if labelT == "Ankle Boot":
                    i = i + 1
                    print("FOUND Ankle Boot")
                # extract the image from the testData if using "channels_first"
                # ordering
                if K.image_data_format() == "channels_first":
                    image = (testX[i][0] * 255).astype("uint8")

                # otherwise we are using "channels_last" ordering
                else:
                    image = (testX[i] * 255).astype("uint8")

                image = cv2.merge([image] * 3)
                image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
                color = (0, 255, 0)
                cv2.putText(image, labelT, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)

                # add the image to our list of output images
                images.append(image)

                # construct the montage for the images
                montage = build_montages(images, (96, 96), (4, 4))[0]

                # show the output montage
                path = os.getcwd() + "\\" + str(i) + "DressBoot.png"
                cv2.imwrite(path, montage)
                images = []
                montage = []
                # cv2.waitKey(0)
            break


trainX, trainY, testX, testY, labelData = getData()
model = train(trainX, trainY, testX, testY, labelData)
makeOutfits(trainX, trainY, testX, testY, labelData, model)

[INFO] loading Fashion MNIST...
[INFO] compiling model...
[]
[INFO] training model...
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 210s 3ms/sample - loss: 0.5177 - accuracy: 0.8192 - val_loss: 0.3481 - val_accuracy: 0.8698
Epoch 2/5
60000/60000 [==============================] - 209s 3ms/sample - loss: 0.3530 - accuracy: 0.8724 - val_loss: 0.2890 - val_accuracy: 0.8958
Epoch 3/5
60000/60000 [==============================] - 210s 3ms/sample - loss: 0.3208 - accuracy: 0.8843 - val_loss: 0.2749 - val_accuracy: 0.9008
Epoch 4/5
60000/60000 [==============================] - 210s 4ms/sample - loss: 0.3001 - accuracy: 0.8924 - val_loss: 0.2625 - val_accuracy: 0.9049
Epoch 5/5
60000/60000 [==============================] - 213s 4ms/sample - loss: 0.2876 - accuracy: 0.8957 - val_loss: 0.2561 - val_accuracy: 0.9055
sandal
sneaker
ankle boot
dress
